### AIM : To Create the model for sentiment analysis using deep learning .

In [ ]:
# importing libraries

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

import re

import pandas as pd
import numpy as np

# implement ann 
from keras import Sequential
from keras.layers import Dense, Embedding, LSTM, SimpleRNN, GRU, Dropout
import tensorflow as tf

In [ ]:
dataframe = pd.read_csv("IMDB Dataset.csv") # importing the dataset 

In [ ]:
dataframe.head() 

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [4]:
dataframe.shape

(50000, 2)

In [ ]:
positive_data = dataframe[dataframe['sentiment'] == 'positive'] # positive comments
negetive_review = dataframe[dataframe['sentiment'] == 'negative'] # negetive comments 

In [ ]:
# selecting 10000 rows for training

training = pd.concat([positive_data.iloc[:45000], negetive_review.iloc[:45000]], axis=0)
test = pd.concat([positive_data.iloc[45000:], negetive_review.iloc[45000:]], axis=0)

print(f"training shape : {training.shape} , testing shape : {test.shape}")


training shape : (50000, 2) , testing shape : (0, 2)


In [7]:
training['sentiment'].unique()

array(['positive', 'negative'], dtype=object)

In [8]:
stop_words =  set(stopwords.words('english'))

In [9]:
def remove_stop_words(text):
    # function to remove the stopwords .

    text = word_tokenize(text)
    result = []

    for i in text :
        if i not in stop_words :
            result.append(i)

    return " ".join(result)

training['review'] = training['review'].apply(remove_stop_words)

In [ ]:
def remove_unwanted_text(text):

    text = re.sub(r"[^a-zA-Z0-9\s]", "", text)  # Keep only letters and spaces
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

training['review'] = training['review'].apply(remove_unwanted_text)

In [11]:
training['review'] = training['review'].apply( lambda x : x.lower())
training['sentiment'].value_counts()

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [12]:
# training['sentiment'] = training['sentiment'].apply(lambda x : 1 if 'positive' else 0)

In [13]:
def transform_review(text):
    if text == 'positive':
        return 1
    else:
        return 0
    
training['sentiment'] = training['sentiment'].apply(transform_review)

In [14]:
training.head()

,review,sentiment
0,one reviewers mentioned watching oz episode ll...,1
1,a wonderful little production br br the filmin...,1
2,i thought wonderful way spend time hot summer ...,1
4,petter mattei s love time money visually stunn...,1
5,probably alltime favorite movie story selfless...,1


### implement Bag-of-words

In [15]:
# max sentence length
max(training['review'].apply(lambda x : len(x.split())))

1532

In [16]:
vectorizer = CountVectorizer(max_features=1000)

x = vectorizer.fit_transform(training['review']).toarray()
y = np.array(training['sentiment'], dtype=np.float32)


In [17]:
x_train, x_test , y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.2)
print(f'x_train : {x_train.shape}, y_train : {y_train.shape}, x_test : {x_test.shape}, y_test : {y_test.shape}')

x_train : (40000, 1000), y_train : (40000,), x_test : (10000, 1000), y_test : (10000,)


### implement model

In [ ]:
# ann model

input_dim = x_train.shape[1]

model = Sequential([
    Dense(16, activation='relu', input_dim=input_dim),
    Dropout(0.3),  # Drops 30% of neurons
    Dense(8, activation='relu'),
    Dropout(0.3),  # Drops 30% of neurons
    Dense(1, activation='sigmoid')
])

with tf.device('/GPU:0'):
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    model.fit(x_train, y_train, epochs=100, batch_size=100, verbose=1)
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    print(f"ANN Accuracy: {accuracy:.4f}")


Epoch 1/100
400/400 [==============================] - 3s 3ms/step - loss: 0.5001 - accuracy: 0.7565
Epoch 2/100
400/400 [==============================] - 1s 4ms/step - loss: 0.3609 - accuracy: 0.8507
Epoch 3/100
400/400 [==============================] - 2s 4ms/step - loss: 0.3382 - accuracy: 0.8636
Epoch 4/100
400/400 [==============================] - 1s 4ms/step - loss: 0.3246 - accuracy: 0.8705
Epoch 5/100
400/400 [==============================] - 2s 4ms/step - loss: 0.3095 - accuracy: 0.8774
Epoch 6/100
400/400 [==============================] - 1s 4ms/step - loss: 0.3033 - accuracy: 0.8798
Epoch 7/100
400/400 [==============================] - 1s 4ms/step - loss: 0.2921 - accuracy: 0.8854
Epoch 8/100
400/400 [==============================] - 1s 3ms/step - loss: 0.2823 - accuracy: 0.8898
Epoch 9/100
400/400 [==============================] - 1s 4ms/step - loss: 0.2746 - accuracy: 0.8924
Epoch 10/100
400/400 [==============================] - 1s 4ms/step - loss: 0.2684 - accura

In [ ]:
# lstm 

model = Sequential([
    Embedding(input_dim=input_dim, output_dim=10, input_length=input_dim),
    LSTM(32, return_sequences=True),
    LSTM(16),
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=20, verbose=1)
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"LSTM Accuracy: {accuracy:.4f}")

In [ ]:
# rnn 

model = Sequential([
        Embedding(input_dim=input_dim, output_dim=10, input_length=input_dim),
        SimpleRNN(16, return_sequences=True),
        SimpleRNN(8),
        Dense(1, activation='sigmoid')
    ])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

with tf.device('/GPU:0'):
    model.fit(x_train, y_train, epochs=10, batch_size=2000, verbose=1)
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    print(f"RNN Accuracy: {accuracy:.4f}")

In [ ]:
# gru 

model = Sequential([
        Embedding(input_dim=input_dim, output_dim=10, input_length=input_dim),
        GRU(16, return_sequences=True),
        GRU(8),
        Dense(1, activation='sigmoid')
    ])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
with tf.device('/GPU:0'):
    model.fit(x_train, y_train, epochs=10, batch_size=200, verbose=1)
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    print(f"GRU Accuracy: {accuracy:.4f}")

In [ ]:
model.save('model.h5') # exporting model 

INFO:tensorflow:Assets written to: ram://47048aef-6e31-4a65-8d2f-f1406202ffec/assets


In [ ]:
# exporting vectorizer 

import pickle as pkl

with open('vectorizer.pkl', 'wb') as f:
    pkl.dump(vectorizer, f)